In [1]:
import datetime

from script.core.transformers import *
from script.core.models import *
from script.core.dataset import Watcher
from script.core.utils import read_dataset, get_result, logging
from script.core.pipeline import Pipeline

# linear models
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# sklearn feachure extractors
from sklearn.feature_extraction.text import CountVectorizer as count
from sklearn.feature_extraction.text import TfidfVectorizer as tfidf
from sklearn.preprocessing import LabelEncoder

# Pure DilyaraModel
from intent_classifier.intent_model.model_wrap import KerasMulticlassModel as DilyaraModel
from script.models.cnn import CNN
from script.models.dcnn import DCNN

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/mks/envs/intent_script/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
2018-03-25 08:20:19.960 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


# Read csv file and create Dataset

In [2]:
def init_dataset():
    path = './data/russian/data/vkusvill_all_categories.csv'
    global_data = read_dataset(path)
    date = datetime.datetime.now()
    dataset = Watcher(global_data, date, seed=42)
#     dataset = dataset.split([0.1, 0.1])

#     print(dataset.data.keys())
#     print(len(dataset.data['valid']))

#     data = dataset.data['test']
#     dataset = Watcher(data, date, seed=42)
#     print(dataset.data.keys())
    
    return dataset

# Operations

In [3]:
spl_conf = {'op_type': 'transformer',
            'name': 'Speller',
            'request_names': ['base'],
            'new_names': ['base'],
            'path': './DeepPavlov/deeppavlov/configs/error_model/brillmoore_kartaslov_ru.json'}

tok_conf = {'op_type': 'transformer',
            'name': 'Tokenizer',
            'request_names': ['base'],
            'new_names': ['base']}

lem_conf = {'op_type': 'transformer',
            'name': 'Lemmatizer',
            'request_names': ['base'],
            'new_names': ['base']}

concat = TextConcat()

tfidf_conf_1 = {'op_type': 'vectorizer', 'name': 'tf-idf vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_conf_2 = {'op_type': 'vectorizer', 'name': 'tf-idf_vectorizer',
                'request_names': ['train', 'valid', 'test'], 'new_names': ['train_vec', 'valid_vec', 'test_vec']}
tfidf_ = sktransformer(tfidf, tfidf_conf_1)

# Pipeline with Neural Models

## Run

In [5]:
class GetCNN(BaseModel):
    def init_model(self, dataset):
        classes = dataset.get_classes()
        classes = ' '.join([str(x) for x in classes])
        self.config['classes'] = classes
        
        super().init_model(dataset)
        
        return self

def get_cnn_conf(path, emb_path, fit_names=None, predict_names=None, new_names=None):
    with open(path, 'r') as conf:
        config = json.load(conf)

    config['op_type'] = 'model'
    config['name'] = 'cnn'
    
    if fit_names is not None:
        config['fit_names'] = fit_names
    else:
        config['fit_names'] = ['train_vec']
    
    if predict_names is not None:
        config['predict_names'] = predict_names
    else:
        config['predict_names'] = ['test_vec']
        
    if new_names is not None:
        config['new_names'] = new_names
    else:
        config['new_names'] = ['predicted_test']
    
    config['fasttext_model'] = emb_path
    
    return config

path_0 = './configs/models/CNN/CNN_opt.json'
path_1 = './configs/models/CNN/cnn.json'
emb_path = './data/russian/embeddings/ft_0.8.3_nltk_yalen_sg_300.bin'

In [5]:
# config = get_cnn_conf(path_0, emb_path, fit_names=['train'], predict_names=['test'])
# dataset = init_dataset()
# Dilyara = GetCNN(DilyaraModel, config)
# neuro_pipe = [(Tokenizer, ), (Lemmatizer,), (TextConcat,), (Dilyara,), (GetResult,)]
# pipeline_0 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_0.run(dataset)

In [23]:
config = get_cnn_conf(path_1, emb_path)
dataset = init_dataset()
WCNN = GetCNN(CNN, config)
neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,)]
pipeline_1 = Pipeline(neuro_pipe, mode='infer', output='dataset')
neurodata = pipeline_1.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 12%|█▏        | 524/4475 [00:00<00:00, 5236.04it/s]

dict_keys(['train', 'valid', 'test'])
4475
dict_keys(['base'])
[ Starting tokenization ... ]


  0%|          | 0/3599 [00:00<?, ?it/s]

[ Tokenization was done. ]
[ Starting vectorization ... ]
[ Vectorization of train part of dataset ... ]


 38%|███▊      | 168/438 [00:00<00:00, 1677.00it/s]

[ Vectorization of valid part of dataset ... ]


 35%|███▍      | 152/438 [00:00<00:00, 1493.57it/s]

[ Vectorization of test part of dataset ... ]


100%|██████████| 438/438 [00:00<00:00, 1555.14it/s]


[ Vectorization was ended. ]
[ Initializing intent_model from scratch ]

____Training over 3599 samples____


train -->	updates: 1	loss: 0.36509931087493896	fmeasure: 0.0	 
epochs_done: 1
train -->	updates: 58	loss: 0.2574595808982849	fmeasure: 0.4337348937988281	 
epochs_done: 2


In [24]:
neurodata.data.keys()

dict_keys(['train', 'valid', 'test', 'train_vec', 'valid_vec', 'test_vec', 'predicted_test'])

In [18]:
a = neurodata.data['test_vec']['report']
a_ = np.zeros((len(a), len(a[0])))
for i, x in enumerate(a):
    for j, y in enumerate(x):
        a_[i][j] = y
a_.shape

(439, 17)

In [21]:
b = np.argmax(a_, axis=1)
b

array([ 2, 12,  2,  0,  5, 14, 11,  2,  5,  2,  5,  2,  2, 15,  5,  1,  5,
        5,  1,  0,  5,  1,  1,  2,  5,  1,  5,  5,  2,  9,  2,  0,  2,  9,
        1,  9,  7,  0,  5,  1, 10,  6, 10,  6,  1,  0,  9,  9, 12,  3,  2,
       11,  5,  1,  5,  8,  0, 12,  5,  2,  9,  5, 15,  1, 12,  8,  5,  0,
        5,  2,  2,  5,  1,  9,  2,  2,  5,  9, 11,  0,  5,  9,  0, 14,  0,
        6,  5, 12,  0,  5, 12,  1,  5,  9,  2,  0, 12,  2,  2,  5, 14,  5,
       12,  5, 10,  6,  6,  5,  6,  2, 12,  7,  6,  9,  5,  6,  1,  6,  5,
        9,  1, 14,  1,  0,  9,  2,  0,  2,  1,  1,  1, 15,  5,  9,  5, 12,
        9,  2,  1,  0,  2, 11,  6,  0,  2,  2, 11, 10,  9,  6,  5,  1,  2,
        5,  2,  5,  1,  2,  5,  5,  5,  3,  7,  1,  5,  0,  3, 10, 12,  6,
        5,  3, 15,  5,  5,  6,  5,  2, 10,  1,  2, 12,  0,  5,  5,  5, 12,
       13,  6,  2,  5, 12,  5,  0,  2, 12, 10,  6,  2,  5,  6,  1,  5,  0,
       12,  5,  2, 11,  8,  5,  5, 10,  5,  5, 14,  3, 10,  5,  9,  1,  3,
        5,  1,  6, 10, 12

In [22]:
c = np.array(neurodata.data['test']['report'])
c

array([ 3, 13,  3,  1,  6, 15, 12,  3,  6,  3,  6,  3,  3, 16,  6,  2,  6,
        6,  2,  1,  6,  2,  2,  3,  6,  2,  6,  6,  3, 10,  3,  1,  3, 10,
        2, 10,  8,  1,  6,  2, 11,  7, 11,  7,  2,  1, 10, 10, 13,  4,  3,
       12,  6,  2,  6,  9,  1, 13,  6,  3, 10,  6, 16,  2, 13,  9,  6,  1,
        6,  3,  3,  6,  2, 10,  3,  3,  6, 10, 12,  1,  6, 10,  1, 15,  1,
        7,  6, 13,  1,  6, 13,  2,  6, 10,  3,  1, 13,  3,  3,  6, 15,  6,
       13,  6, 11,  7,  7,  6,  7,  3, 13,  8,  7, 10,  6,  7,  2,  7,  6,
       10,  2, 15,  2,  1, 10,  3,  1,  3,  2,  2,  2, 16,  6, 10,  6, 13,
       10,  3,  2,  1,  3, 12,  7,  1,  3,  3, 12, 11, 10,  7,  6,  2,  3,
        6,  3,  6,  2,  3,  6,  6,  6,  4,  8,  2,  6,  1,  4, 11, 13,  7,
        6,  4, 16,  6,  6,  7,  6,  3, 11,  2,  3, 13,  1,  6,  6,  6, 13,
       14,  7,  3,  6, 13,  6,  1,  3, 13, 11,  7,  3,  6,  7,  2,  6,  1,
       13,  6,  3, 12,  9,  6,  6, 11,  6,  6, 15,  4, 11,  6, 10,  2,  4,
        6,  2,  7, 11, 13

In [25]:
pred_data = neurodata.data['predicted_test']

preds = pred_data[0]
for x in pred_data[1:]:
    preds = np.concatenate((preds, x), axis=0)

In [26]:
preds.shape

(448, 17)

In [27]:
preds = np.argmax(preds, axis=1)
print(preds.shape)
preds

(448,)


array([12, 12,  0, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12,  1, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12,  6, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        6, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12,  1, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12,  0, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,  0, 12,
       12,  6, 12, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12

In [6]:
config = get_cnn_conf(path_1, emb_path)
dataset = init_dataset()
WCNN = GetCNN(CNN, config)
neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (WCNN,), (GetResult,)]
pipeline_1 = Pipeline(neuro_pipe, mode='infer', output='dataset')
neurodata = pipeline_1.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
  2%|▏         | 1032/45171 [00:00<00:08, 5156.20it/s]

[ Starting tokenization ... ]


100%|██████████| 45171/45171 [00:09<00:00, 4960.09it/s]


[ Tokenization was done. ]


  1%|          | 188/36221 [00:00<00:19, 1877.96it/s]

...

  4%|▎         | 165/4475 [00:00<00:02, 1648.63it/s]

[ Vectorization of test part of dataset ... ]


100%|██████████| 4475/4475 [00:02<00:00, 1722.52it/s]


[ Vectorization was ended. ]
[ Initializing intent_model from scratch ]

____Training over 36221 samples____


train -->	updates: 1	loss: 0.35727494955062866	fmeasure: 0.0	 
train -->	updates: 501	loss: 0.15314698219299316	fmeasure: 0.5957446098327637	 
epochs_done: 1
train -->	updates: 567	loss: 0.1640733778476715	fmeasure: 0.5208332538604736	 
train -->	updates: 1067	loss: 0.15857265889644623	fmeasure: 0.4905659556388855	 
epochs_done: 2


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
# config = get_cnn_conf(path_1, emb_path)
# dataset = init_dataset()
# dCNN = GetCNN(DCNN, config)
# neuro_pipe = [(Tokenizer, ), (FasttextVectorizer,), (dCNN,), (GetResult,)]
# pipeline_2 = Pipeline(neuro_pipe, mode='infer', output='dataset')
# neurodata = pipeline_2.run(dataset)

# Pipelines with Linear Models

In [8]:
conf_0 = {'op_type': 'model', 'name': 'Linear Regression',
          'fit_names': ['train_vec'], 'new_names': ['predicted_test'],
          'predict_names': ['test_vec']}

## Run pipeline with linear regression

In [9]:
# (Tokenizer, tok_conf), (Lemmatizer,), (concat, None),
dataset = init_dataset()
LR = skmodel(LogisticRegression, conf_0)

pipe_0 = [(tfidf_, tfidf_conf_2), (LR,), (GetResultLinear_W,)]
pipeline_0 = Pipeline(pipe_0, mode='train', output=None)
pipeline_1 = Pipeline(pipe_0, mode='infer', output='dataset')
res = pipeline_1.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


dict_keys(['train', 'valid', 'test'])
4475
dict_keys(['base'])


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Run pipeline with SVC

In [10]:
dataset = init_dataset()

conf_0['name'] = 'LinearSVC'
conf_0['op_type'] = 'model'
LSVC = skmodel(LinearSVC, conf_0)

# (Speller, spl_conf),

pipe_1 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
          (LSVC,), (GetResultLinear_W,)]
pipeline_2 = Pipeline(pipe_1, mode='train', output=None)
pipeline_3 = Pipeline(pipe_1, mode='infer', output='dataset')

res = pipeline_3.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 12%|█▏        | 538/4475 [00:00<00:00, 5374.83it/s]

dict_keys(['train', 'valid', 'test'])
4475
dict_keys(['base'])
[ Starting tokenization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 5276.25it/s]
2018-03-25 07:17:59.720 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/mks/envs/intent_script/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-03-25 07:17:59.749 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
  0%|          | 16/4475 [00:00<00:29, 153.31it/s]

[ Tokenization was done. ]
[ Starting lemmatization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 440164.87it/s]
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


[ Ended lemmatization. ]
[ Starting text merging ... ]
[ Text concatenation was ended. ]


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


##  Run pipeline with Random Forest

In [11]:
dataset = init_dataset()

conf_0['name'] = 'RandomForestClassifier'
conf_0['op_type'] = 'model'
RFC = skmodel(RandomForestClassifier, conf_0)


# (Speller, spl_conf), 
pipe_2 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
          (RFC,), (GetResultLinear_W,)]
pipeline_4 = Pipeline(pipe_2, mode='train', output=None)
pipeline_5 = Pipeline(pipe_2, mode='infer', output='dataset')

res = pipeline_5.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 12%|█▏        | 524/4475 [00:00<00:00, 5217.87it/s]

dict_keys(['train', 'valid', 'test'])
4475
dict_keys(['base'])
[ Starting tokenization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 5327.62it/s]
2018-03-25 07:18:30.321 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/mks/envs/intent_script/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-03-25 07:18:30.350 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
  0%|          | 0/4475 [00:00<?, ?it/s]

[ Tokenization was done. ]
[ Starting lemmatization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 451950.65it/s]
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


[ Ended lemmatization. ]
[ Starting text merging ... ]
[ Text concatenation was ended. ]


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Run pipeline with GBM

In [12]:
dataset = init_dataset()

conf_0['name'] = 'LGBMClassifier'
conf_0['op_type'] = 'model'
LGBM = skmodel(LGBMClassifier, conf_0)

# (Speller, spl_conf),
pipe_3 = [(Tokenizer, ), (Lemmatizer,), (concat, None), (tfidf_, tfidf_conf_2),
          (LGBM,), (GetResultLinear_W,)]
pipeline_6 = Pipeline(pipe_3, mode='train', output=None)
pipeline_7 = Pipeline(pipe_3, mode='infer', output='dataset')

res = pipeline_7.run(dataset)

/home/mks/envs/intent_script/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
 11%|█         | 494/4475 [00:00<00:00, 4939.40it/s]

dict_keys(['train', 'valid', 'test'])
4475
dict_keys(['base'])
[ Starting tokenization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 5269.91it/s]
2018-03-25 07:19:01.60 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /home/mks/envs/intent_script/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-03-25 07:19:01.89 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
  0%|          | 0/4475 [00:00<?, ?it/s]

[ Tokenization was done. ]
[ Starting lemmatization ... ]


100%|██████████| 4475/4475 [00:00<00:00, 442186.97it/s]


[ Ended lemmatization. ]
[ Starting text merging ... ]
[ Text concatenation was ended. ]


/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mks/envs/intent_script/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: